In [1]:
from tqdm.auto import tqdm

from datasets import load_dataset, load_metric
from datasets import ClassLabel, Sequence
from transformers import AutoTokenizer, PreTrainedTokenizerFast
from transformers import AutoModelForQuestionAnswering
from transformers import default_data_collator
from transformers import get_scheduler
from transformers import pipeline
import transformers
from accelerate import Accelerator

from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch
import random
import pandas as pd


print('transformers', transformers.__version__)
print('torch', torch.version.__version__)

2023-01-27 12:27:56.482305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


transformers 4.26.0
torch 1.13.0a0+git49444c3


In [2]:
datasets = load_dataset("squad")
datasets

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [3]:
datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [4]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    return df


show_random_elements(datasets["train"])

,id,title,context,question,answers
0,56d13443e7d4791d00901feb,IPod,"On June 11, 2006, the British tabloid The Mail...",Which British tabloid accused Apple of unfair ...,"{'text': ['The Mail'], 'answer_start': [38]}"
1,57293c5e6aef051400154bc0,Alaska,Alaska's internet and other data transport sys...,Which two companies provide internet and data ...,"{'text': ['GCI and Alaska Communications'], 'a..."
2,572bbb82111d821400f38f64,"New_Haven,_Connecticut",Yale and New Haven are working to build a medi...,Although some of the Science Park area is used...,{'text': ['parking lots or abandoned structure...
3,56f82554aef2371900625e28,Josip_Broz_Tito,In the years following the dissolution of Yugo...,Under which ruler do historians argue human ri...,"{'text': ['Tito'], 'answer_start': [131]}"
4,570b2f5e6b8089140040f7ea,Gregorian_calendar,The Gregorian calendar improves the approximat...,"What is the approximate error for every 3,300 ...","{'text': ['one day'], 'answer_start': [231]}"
5,573366074776f419006609e5,Alfred_North_Whitehead,Whitehead thus sees God and the world as fulfi...,How dis Whitehead believe God provided permane...,"{'text': ['by taking them into God's self, the..."
6,572632a1ec44d21400f3dc29,Queen_Victoria,"Internationally, Victoria took a keen interest...",Which groups were demonstrating and trying to ...,"{'text': ['Chartists and Irish nationalists'],..."
7,570df7950b85d914000d7c31,Antarctica,"As a result of continued warming, the polar ic...",During what period did Gondwana begin to break...,"{'text': ['Cretaceous'], 'answer_start': [674]}"
8,5727d5254b864d1900163e25,Detroit,"Oakland County in Metro Detroit, once rated am...",What city dominates Wayne County?,"{'text': ['Detroit'], 'answer_start': [431]}"
9,56e0f6177aa994140058e829,Space_Race,Gherman Titov became the first Soviet cosmonau...,What first person from the USSR controlled the...,"{'text': ['Gherman Titov'], 'answer_start': [0]}"


In [5]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

assert isinstance(tokenizer, PreTrainedTokenizerFast)

In [6]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

tokenizer `__call__` will insert `[CLS]` at the start of the `input_ids` and a `[SEP]` in between the sentence and an extra `[SEP]` at the end of the sentence

In [7]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

We don't want to split the `question`, but we could split the `context`

In [8]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [9]:
len(tokenizer(example["question"], example["context"])["input_ids"])

396

In [10]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [11]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

use `only_second` to truncation the second input text

add `return_overflowing_tokens` so the tokenizer will return a list of list of ids when the input text exceed `max_length`

`stride` (int, optional, defaults to 0) — If set to a number along with `max_length`, the overflowing tokens returned when `return_overflowing_tokens=True` will contain some tokens from the end of the truncated sequence returned to provide some overlap between truncated and overflowing sequences. The value of this argument defines the number of overlapping tokens.

In [12]:
tokenized_example

{'input_ids': [[101, 2129, 2116, 5222, 2515, 1996, 10289, 8214, 2273, 1005, 1055, 3455, 2136, 2031, 1029, 102, 1996, 2273, 1005, 1055, 3455, 2136, 2038, 2058, 1015, 1010, 5174, 5222, 1010, 2028, 1997, 2069, 2260, 2816, 2040, 2031, 2584, 2008, 2928, 1010, 1998, 2031, 2596, 1999, 2654, 5803, 8504, 1012, 2280, 2447, 5899, 12385, 4324, 1996, 2501, 2005, 2087, 2685, 3195, 1999, 1037, 2309, 2208, 1997, 1996, 2977, 2007, 6079, 1012, 2348, 1996, 2136, 2038, 2196, 2180, 1996, 5803, 2977, 1010, 2027, 2020, 2315, 2011, 1996, 16254, 2015, 5188, 3192, 2004, 2120, 3966, 3807, 1012, 1996, 2136, 2038, 23339, 1037, 2193, 1997, 6314, 2015, 1997, 2193, 2028, 4396, 2780, 1010, 1996, 2087, 3862, 1997, 2029, 2001, 4566, 12389, 1005, 1055, 2501, 6070, 1011, 2208, 3045, 9039, 1999, 3326, 1012, 1996, 2136, 2038, 7854, 2019, 3176, 2809, 2193, 1011, 2028, 2780, 1010, 1998, 2216, 3157, 5222, 4635, 2117, 1010, 2000, 12389, 1005, 1055, 2184, 1010, 2035, 1011, 2051, 1999, 5222, 2114, 1996, 2327, 2136, 1012, 1996, 21

In [13]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 157]

In [14]:
for x in tokenized_example["input_ids"]:
    print(len(x))
    print(tokenizer.decode(x))

384
[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth at 

add `return_offsets_mapping` for tokenizer to return `offsets_mapping`, so it is easier to find the `answer`'s position

In [15]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 37), (37, 38), (38, 39), (40, 50), (51, 55), (56, 60), (60, 61), (0, 0), (0, 3), (4, 7), (7, 8), (8, 9), (10, 20), (21, 25), (26, 29), (30, 34), (35, 36), (36, 37), (37, 40), (41, 45), (45, 46), (47, 50), (51, 53), (54, 58), (59, 61), (62, 69), (70, 73), (74, 78), (79, 86), (87, 91), (92, 96), (96, 97), (98, 101), (102, 106), (107, 115), (116, 118), (119, 121), (122, 126), (127, 138), (138, 139), (140, 146), (147, 153), (154, 160), (161, 165), (166, 171), (172, 175), (176, 182), (183, 186), (187, 191), (192, 198), (199, 205), (206, 208), (209, 210), (211, 217), (218, 222), (223, 225), (226, 229), (230, 240), (241, 245), (246, 248), (248, 249), (250, 258), (259, 262), (263, 267), (268, 271), (272, 277), (278, 281), (282, 285), (286, 290), (291, 301), (301, 302), (303, 307), (308, 312), (313, 318), (319, 321), (322, 325), (326, 330), (330, 331), (332, 340), (341, 351), (352, 354), (355, 363), (364, 373), (374,

In [16]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

how How


In [17]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

with `offset_mapping`, it is easier to find the `answer`'s position

In [18]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

23 26


In [19]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

over 1, 600
over 1,600


In [20]:
pad_on_right = tokenizer.padding_side == "right"

In [21]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


features = prepare_train_features(datasets['train'][:5])
features

{'input_ids': [[101, 2000, 3183, 2106, 1996, 6261, 2984, 9382, 3711, 1999, 8517, 1999, 10223, 26371, 2605, 1029, 102, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, 23052, 2012, 10223, 26371, 1010, 2605, 2073, 1996, 6261, 2984, 22353, 2135, 2596, 2000, 3002, 16595, 9648, 4674, 2061, 12083, 9711, 2271, 1999, 8517, 1012, 2012, 1996, 2203, 1997, 1996, 2364, 3298, 1006, 1998, 1999, 1037, 3622, 2240, 2008, 8539, 2083, 1017, 11342, 1998, 1996, 2

In [22]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

In [23]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [24]:
data_collator = default_data_collator
batch_size = 16
learning_rate = 2e-5
epochs = 3

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=batch_size, collate_fn=data_collator
)

In [25]:
accelerator = Accelerator()
optimizer = AdamW(model.parameters(), lr=learning_rate)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [26]:
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [27]:
progress_bar = tqdm(range(num_training_steps))
output_dir = f'{model_name}-squad'

for epoch in range(epochs):
    # Training
    model.train()
    losses = []
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        
        losses.append(loss)

    print(f">>> Epoch {epoch}: loss: {torch.mean(torch.stack(losses))}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

  0%|          | 0/16599 [00:00<?, ?it/s]

>>> Epoch 0: loss: 1.4935489892959595
>>> Epoch 1: loss: 0.9560167789459229
>>> Epoch 2: loss: 0.7503600716590881


In [28]:
model.to('cpu')
question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [29]:
datasets["validation"][0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [30]:
val_example = datasets["validation"][0]
question = val_example['question']
context = val_example['context']
n_best_size = 20

print('context:', context)
print('question:', question)
question_answerer(
    question=question,
    context=context,
    topk=n_best_size
)

context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
question: Which NFL team represented the AFC at Super Bowl 50?


/home/kiddos/.local/lib/python3.8/site-packages/transformers/pipelines/question_answering.py:325: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)


[{'score': 0.4742189645767212,
  'start': 177,
  'end': 191,
  'answer': 'Denver Broncos'},
 {'score': 0.06369159370660782, 'start': 184, 'end': 191, 'answer': 'Broncos'},
 {'score': 0.051203977316617966,
  'start': 177,
  'end': 266,
  'answer': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 0.03725113347172737,
  'start': 249,
  'end': 266,
  'answer': 'Carolina Panthers'},
 {'score': 0.00920463353395462,
  'start': 129,
  'end': 191,
  'answer': 'The American Football Conference (AFC) champion Denver Broncos'},
 {'score': 0.006877124775201082,
  'start': 184,
  'end': 266,
  'answer': 'Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 0.0013062362559139729,
  'start': 133,
  'end': 191,
  'answer': 'American Football Conference (AFC) champion Denver Broncos'},
 {'score': 0.0008745072409510612,
  'start': 177,
  'end': 183,
  'answer': 'Denver'},
 {'score': 0.0005387251148931682,


In [31]:
val_example = datasets["validation"][500]
question = val_example['question']
context = val_example['context']

print('context:', context)
print('question:', question)
question_answerer(
    question=question,
    context=context,
    topk=n_best_size
)

context: The game's media day, which was typically held on the Tuesday afternoon prior to the game, was moved to the Monday evening and re-branded as Super Bowl Opening Night. The event was held on February 1, 2016 at SAP Center in San Jose. Alongside the traditional media availabilities, the event featured an opening ceremony with player introductions on a replica of the Golden Gate Bridge.
question: What was media day called for Super Bowl 50?


[{'score': 0.9453121423721313,
  'start': 141,
  'end': 165,
  'answer': 'Super Bowl Opening Night'},
 {'score': 0.0047735972329974174,
  'start': 152,
  'end': 165,
  'answer': 'Opening Night'},
 {'score': 0.0034648508299142122,
  'start': 141,
  'end': 151,
  'answer': 'Super Bowl'},
 {'score': 0.0011605884646996856,
  'start': 141,
  'end': 166,
  'answer': 'Super Bowl Opening Night.'},
 {'score': 0.0008657319121994078,
  'start': 108,
  'end': 165,
  'answer': 'Monday evening and re-branded as Super Bowl Opening Night'},
 {'score': 0.0005988003686070442, 'start': 160, 'end': 165, 'answer': 'Night'},
 {'score': 0.00019675932708196342,
  'start': 147,
  'end': 165,
  'answer': 'Bowl Opening Night'},
 {'score': 0.00019351202354300767,
  'start': 141,
  'end': 205,
  'answer': 'Super Bowl Opening Night. The event was held on February 1, 2016'},
 {'score': 0.00019088745466433465,
  'start': 138,
  'end': 165,
  'answer': 'as Super Bowl Opening Night'},
 {'score': 0.00013066908286418766,

In [32]:
for entry in datasets['validation']:
    if len(entry['question']) > 500:
        break

val_example = entry
question = val_example['question']
context = val_example['context']

print('context:', context)
print('question:', question)
question_answerer(
    question=question,
    context=context,
    topk=n_best_size
)

context: The pound-force has a metric counterpart, less commonly used than the newton: the kilogram-force (kgf) (sometimes kilopond), is the force exerted by standard gravity on one kilogram of mass. The kilogram-force leads to an alternate, but rarely used unit of mass: the metric slug (sometimes mug or hyl) is that mass that accelerates at 1 m·s−2 when subjected to a force of 1 kgf. The kilogram-force is not a part of the modern SI system, and is generally deprecated; however it still sees use for some purposes as expressing aircraft weight, jet thrust, bicycle spoke tension, torque wrench settings and engine output torque. Other arcane units of force include the sthène, which is equivalent to 1000 N, and the kip, which is equivalent to 1000 lbf.
question: What is the seldom used force unit equal to one thousand newtons?


[{'score': 0.10045231878757477, 'start': 708, 'end': 715, 'answer': 'the kip'},
 {'score': 0.04036340117454529, 'start': 712, 'end': 715, 'answer': 'kip'},
 {'score': 0.022352244704961777,
  'start': 708,
  'end': 748,
  'answer': 'the kip, which is equivalent to 1000 lbf'},
 {'score': 0.013230492360889912,
  'start': 665,
  'end': 715,
  'answer': 'sthène, which is equivalent to 1000 N, and the kip'},
 {'score': 0.012182257138192654,
  'start': 708,
  'end': 749,
  'answer': 'the kip, which is equivalent to 1000 lbf.'},
 {'score': 0.008981500752270222,
  'start': 712,
  'end': 748,
  'answer': 'kip, which is equivalent to 1000 lbf'},
 {'score': 0.008532815612852573,
  'start': 263,
  'end': 278,
  'answer': 'the metric slug'},
 {'score': 0.007016087882220745,
  'start': 267,
  'end': 278,
  'answer': 'metric slug'},
 {'score': 0.00489503238350153,
  'start': 712,
  'end': 749,
  'answer': 'kip, which is equivalent to 1000 lbf.'},
 {'score': 0.00420584948733449,
  'start': 740,
  'end'